## Weather Watcher: Real-Time Weather Monitoring and Alert System using python

In [2]:
import sqlite3
from datetime import datetime
import requests

# Database setup
conn = sqlite3.connect('weather.db')
cursor = conn.cursor()

# Create Weather table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS Weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    city TEXT NOT NULL,
    date TEXT NOT NULL,
    temperature REAL,
    humidity REAL,
    description TEXT
)
""")
print("Database setup complete.")

# OpenWeatherMap API key
API_KEY = 'e84485d5b239972b148f93de911d9369'
BASE_URL = 'http://api.openweathermap.org/data/2.5/weather'

# Function to fetch weather data and store in database
def fetch_and_store_weather(city):
    params = {'q': city, 'appid': API_KEY, 'units': 'metric'}
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    
    if data['cod'] == 200:
        weather = {
            'city': city,
            'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'temperature': data['main']['temp'],
            'humidity': data['main']['humidity'],
            'description': data['weather'][0]['description']
        }
        
        # Check for conditions that might indicate a tsunami
        if is_tsunami_alert(data):
            alert_tsunami(city)
        
        # Store weather data in database
        cursor.execute("""
            INSERT INTO Weather (city, date, temperature, humidity, description)
            VALUES (?, ?, ?, ?, ?)
        """, (weather['city'], weather['date'], weather['temperature'], weather['humidity'], weather['description']))
        conn.commit()
        
        print(f"Weather data for {city} stored successfully.")
    else:
        print(f"Error fetching data for {city}: {data['message']}")

def is_tsunami_alert(weather_data):
    # Example logic to determine if a tsunami alert should be triggered
    # You might want to check for conditions that could lead to a tsunami,
    # such as certain types of severe weather or earthquake data from other sources.
    # This example just checks if there's a severe weather description.
    if 'storm' in weather_data['weather'][0]['description'].lower():
        return True
    return False

def alert_tsunami(city):
    # Example function to alert about a tsunami
    print(f"Alert: Possible tsunami conditions detected in {city}!")
    # Here you could implement further actions such as sending notifications, etc.
    # For demonstration purposes, it just prints a message.

# Function to view weather data for a city
def view_weather(city):
    cursor.execute("""
        SELECT city, date, temperature, humidity, description
        FROM Weather
        WHERE city = ?
        ORDER BY date DESC
    """, (city,))
    records = cursor.fetchall()
    for record in records:
        print(f"City: {record[0]}, Date: {record[1]}, Temperature: {record[2]}°C, Humidity: {record[3]}%, Description: {record[4]}")

# View weather data for a city
city = input("Enter the city name: ")
fetch_and_store_weather(city)
print(f"\nWeather data for {city}:")
view_weather(city)

# Function to view the latest weather data for all cities
def view_latest_weather():
    cursor.execute("""
        SELECT city, date, temperature, humidity, description
        FROM Weather
        WHERE date IN (
            SELECT MAX(date)
            FROM Weather
            GROUP BY city
        )
        ORDER BY city
    """)
    records = cursor.fetchall()
    for record in records:
        print(f"City: {record[0]}, Date: {record[1]}, Temperature: {record[2]}°C, Humidity: {record[3]}%, Description: {record[4]}")

# View the latest weather data for all cities
print("\nLatest weather data for all cities:")
view_latest_weather()

# Close the connection
conn.close()


Database setup complete.


Enter the city name:  Mandya


Weather data for Mandya stored successfully.

Weather data for Mandya:
City: Mandya, Date: 2024-11-20 17:48:42, Temperature: 25.47°C, Humidity: 76.0%, Description: broken clouds

Latest weather data for all cities:
City: Bengaluru, Date: 2024-11-20 17:36:08, Temperature: 24.31°C, Humidity: 72.0%, Description: mist
City: Mandya, Date: 2024-11-20 17:48:42, Temperature: 25.47°C, Humidity: 76.0%, Description: broken clouds
City: Mysore, Date: 2024-11-20 17:40:58, Temperature: 26.0°C, Humidity: 65.0%, Description: scattered clouds


### This project, Weather Watcher, is a real-time weather monitoring and alert system that integrates the OpenWeatherMap API to fetch,store, and display weather data. It features a SQLite database for efficient data management, allowing users to view current and historical weather information for specific cities. The system includes a basic alert mechanism to detect severe weather conditions that could indicate potential risks, such as tsunamis, ensuring timely awareness and preparedness.

###### coded by Rahul R